# Quantum Ising Model
Following a notebook by [Carsten Bauer & Katharine Hyatt](https://github.com/crstnbr/JuliaOulu20/blob/master/Day2/4_linear_algebra/2_ed_quantum_ising.ipynb).

The [Ising Model](https://en.wikipedia.org/wiki/Ising_model) is a model of ferromagnetism, which considers localized discrete spins of a quantum system.

We will study the Ising model in terms of the time-independent Schrödinger equation
$$
H \left|{ \psi }\right\rangle = E \left|{ \psi }\right\rangle,
$$

where $H$ is to be diagonalized by means of a computational model.

## Transverse field Ising chain
We consider the general Ising hamiltonian function

$$
\mathscr{H} = - \sum_{\left\langle i,j \right\rangle} J_{ij}\sigma_i \sigma_j - \mu \sum_j h_j \sigma_j
$$

for any two adjacent crystal sites $i, j$, with interaction $J_{i,j}$ between sites, and external magnetic field $h_j$. The first sum is across pairs of adjecent spins, with notation $\left\langle i,j \right\rangle$ denoting nearest neighbour.

We will denote the spin states through the Pauli matrices, with the external magnetic field acting perpendicular to the spin orientations -- i.e., we use $\sigma^z$ for spin directions (the first summation term), and $\sigma^x$ for interaction with the magnetic field (second summation term).

We define

In [3]:
σᶻ = [1 0; 0 -1]
σˣ = [0 1; 1 0]

σᶻ, σˣ

([1 0; 0 -1], [0 1; 1 0])

We denote the Eigenvectors of $\sigma^z$ by $\left|{ \uparrow }\right\rangle$, \left|{ \downarrow }\right\rangle respective to their orientation along the $z$ axis -- these will also denote our spin orientations.

We can then see that the function of $\sigma^x$ is to flip the orientation, as it is purely off-diagonal:

In [29]:
using LinearAlgebra

evecs = eigen(σᶻ).vectors
s_down = evecs[1, :]
s_up = evecs[2, :]

@show s_up, s_down

s_up == σˣ * s_down

(s_up, s_down) = ([1.0, 0.0], [0.0, 1.0])


true